In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#format for floats
pd.options.display.float_format = '{:,.8f}'.format

In [2]:
df0 = pd.read_csv('../Neueda2023/Datasets/2010.csv')
df1 = pd.read_csv('../Neueda2023/Datasets/2011.csv')
df2 = pd.read_csv('../Neueda2023/Datasets/2012.csv')
df3 = pd.read_csv('../Neueda2023/Datasets/2013.csv')
df4 = pd.read_csv('../Neueda2023/Datasets/2014.csv')
df5 = pd.read_csv('../Neueda2023/Datasets/2015.csv')
df6 = pd.read_csv('../Neueda2023/Datasets/2016.csv')

In [3]:
df_combined = pd.concat([df0, df1, df2, df3, df4, df5, df6])

In [4]:
!pip install pyxlsb

In [5]:
df_ev = pd.read_excel("./Datasets/vehicle.xlsb", sheet_name="County", engine="pyxlsb", header=1)
df_ev

,state_abbr,geoid,county_id,county_name,fuel_type_org,fuel_type,class,before 1980,1980-99,1990-99,...,2009.1,2010.1,2011.1,2012.1,2013.1,2014.1,2015.1,2016.1,2017.1,2018.1
0,AL,1001,1001,Autauga,BI,Other/Unknown,Car,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AL,1001,1001,Autauga,BI,Other/Unknown,Truck,NaN,NaN,0.00001904,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AL,1001,1001,Autauga,DIES,Diesel vehicle,Car,NaN,0.00019045,0.00003809,...,0.00000571,0.00000000,0.00000950,0.00000728,0.00000867,0.00001158,0.00000502,NaN,NaN,NaN
3,AL,1001,1001,Autauga,DIES,Diesel vehicle,Truck,NaN,0.00114268,0.00544679,...,0.00000000,0.00000000,0.00005713,0.00005112,0.00000000,0.00002943,0.00007849,0.00007428,0.00000000,NaN
4,AL,1001,1001,Autauga,DIES,Diesel vehicle,Unknown,0.00020949,0.00001904,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50810,WY,56045,56045,Weston,GAS,Gasoline vehicle,Unknown,0.04801597,0.00316976,0.00093919,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50811,WY,56045,56045,Weston,HYBRID ELECTRIC VEHICLE,Hybrid electric vehicle,Car,NaN,NaN,NaN,...,0.00000000,0.00000000,0.00000000,0.00000267,0.00001526,0.00000787,0.00000286,0.00000000,0.00000000,NaN
50812,WY,56045,56045,Weston,HYBRID ELECTRIC VEHICLE,Hybrid electric vehicle,Truck,NaN,NaN,NaN,...,NaN,NaN,0.00000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50813,WY,56045,56045,Weston,PLUG IN HYBRID ELECTRIC VEHICLE,Plug-in hybrid electric vehicle,Car,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.00000000,NaN,NaN,0.00000000,NaN,NaN


In [6]:
df_ev = df_ev.iloc[:, [0, 4, 42, 43, 44, 45, 46, 47, 48]]

In [7]:
df_ev = df_ev.fillna(0)

In [8]:
df_ev2 = df_ev.loc[df_ev['fuel_type_org'].isin(['ELECTRIC VEHICLE', 'HYBRID ELECTRIC VEHICLE', 'PLUG IN HYBRID ELECTRIC VEHICLE'])]
df_ev2

,state_abbr,fuel_type_org,2010.1,2011.1,2012.1,2013.1,2014.1,2015.1,2016.1
5,AL,ELECTRIC VEHICLE,0.00000000,0.00000000,0.00000000,0.00000000,0.00000041,0.00000017,0.00000000
12,AL,HYBRID ELECTRIC VEHICLE,0.00000000,0.00000000,0.00001350,0.00002048,0.00001111,0.00001100,0.00000572
13,AL,HYBRID ELECTRIC VEHICLE,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000
14,AL,PLUG IN HYBRID ELECTRIC VEHICLE,0.00000000,0.00000000,0.00000090,0.00000000,0.00000000,0.00000051,0.00000000
24,AL,ELECTRIC VEHICLE,0.00000000,0.00000000,0.00000029,0.00000028,0.00000043,0.00000050,0.00000000
...,...,...,...,...,...,...,...,...,...
50797,WY,PLUG IN HYBRID ELECTRIC VEHICLE,0.00000000,0.00000000,0.00000287,0.00000000,0.00000000,0.00000000,0.00000000
50804,WY,ELECTRIC VEHICLE,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000
50811,WY,HYBRID ELECTRIC VEHICLE,0.00000000,0.00000000,0.00000267,0.00001526,0.00000787,0.00000286,0.00000000
50812,WY,HYBRID ELECTRIC VEHICLE,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000


In [9]:
df_ev2 = pd.melt(df_ev2, id_vars='state_abbr', value_vars=["2010.1", "2011.1", "2012.1", "2013.1", "2014.1", "2015.1", "2016.1"], var_name="Year")

df_ev2["Year"] = df_ev2["Year"].str[:-2].astype(int)

df_ev2

,state_abbr,Year,value
0,AL,2010,0.00000000
1,AL,2010,0.00000000
2,AL,2010,0.00000000
3,AL,2010,0.00000000
4,AL,2010,0.00000000
...,...,...,...
85325,WY,2016,0.00000000
85326,WY,2016,0.00000000
85327,WY,2016,0.00000000
85328,WY,2016,0.00000000


In [10]:
df_ev2 = df_ev2.groupby(["Year", "state_abbr"]).sum()
df_ev2

value
Year state_abbr           
2010 AK         0.00000000
     AL         0.00000000
     AR         0.00000000
     AZ         0.00000000
     CA         0.00000000
...                    ...
2016 VT         0.00060890
     WA         0.00097712
     WI         0.00095506
     WV         0.00041507
     WY         0.00015999

[357 rows x 1 columns]

In [11]:
state_codes = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY"
}

In [12]:
df_combined

,State,County,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days PM2.5,Days PM10
0,Alabama,Baldwin,2010,274,188,80,6,0,0,0,150,71,43,0,0,192,82,0
1,Alabama,Clay,2010,113,79,34,0,0,0,0,86,60,42,0,0,0,113,0
2,Alabama,Colbert,2010,355,208,146,1,0,0,0,105,67,47,0,0,114,241,0
3,Alabama,DeKalb,2010,358,248,106,4,0,0,0,101,74,43,0,0,303,55,0
4,Alabama,Elmore,2010,241,181,55,5,0,0,0,112,71,43,0,0,241,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1025,Wyoming,Sublette,2016,366,297,68,1,0,0,0,102,61,44,0,0,351,15,0
1026,Wyoming,Sweetwater,2016,366,304,62,0,0,0,0,100,58,44,0,0,327,4,35
1027,Wyoming,Teton,2016,366,331,35,0,0,0,0,99,50,41,0,0,351,15,0
1028,Wyoming,Uinta,2016,366,341,25,0,0,0,0,87,49,42,0,0,359,0,7


In [13]:
df_combined["State"] = df_combined["State"].map(state_codes)

In [14]:
df_combined

,State,County,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days PM2.5,Days PM10
0,AL,Baldwin,2010,274,188,80,6,0,0,0,150,71,43,0,0,192,82,0
1,AL,Clay,2010,113,79,34,0,0,0,0,86,60,42,0,0,0,113,0
2,AL,Colbert,2010,355,208,146,1,0,0,0,105,67,47,0,0,114,241,0
3,AL,DeKalb,2010,358,248,106,4,0,0,0,101,74,43,0,0,303,55,0
4,AL,Elmore,2010,241,181,55,5,0,0,0,112,71,43,0,0,241,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1025,WY,Sublette,2016,366,297,68,1,0,0,0,102,61,44,0,0,351,15,0
1026,WY,Sweetwater,2016,366,304,62,0,0,0,0,100,58,44,0,0,327,4,35
1027,WY,Teton,2016,366,331,35,0,0,0,0,99,50,41,0,0,351,15,0
1028,WY,Uinta,2016,366,341,25,0,0,0,0,87,49,42,0,0,359,0,7


In [15]:
df_combined = df_combined.drop("County", axis="columns")

In [16]:
df_combined = df_combined.groupby(['Year', 'State']).mean()

In [17]:
df_combined

Days with AQI    Good Days  Moderate Days  \
Year State                                              
2010 AK      262.12500000 222.00000000    35.00000000   
     AL      282.26315789 164.89473684   113.05263158   
     AR      208.23529412 138.52941176    67.35294118   
     AZ      309.53846154 199.07692308   102.23076923   
     CA      350.39622642 221.94339623   102.33962264   
...                   ...          ...            ...   
2016 VT      301.50000000 279.75000000    21.50000000   
     WA      348.75862069 317.44827586    30.34482759   
     WI      282.92592593 243.25925926    35.85185185   
     WV      272.57142857 237.78571429    34.07142857   
     WY      329.38888889 286.22222222    42.94444444   

            Unhealthy for Sensitive Groups Days  Unhealthy Days  \
Year State                                                        
2010 AK                              4.12500000      1.00000000   
     AL                              4.00000000      0.31578947   
     AR                              2.23529412      0.11764706   
     AZ                              7.92307692      0.15384615   
     CA                             19.88679245      5.32075472   
...                                         ...             ...   
2016 VT                              0.25000000      0.00000000   
     WA                              0.79310345      0.17241379   
     WI                              3.55555556      0.25925926   
     WV                              0.71428571      0.00000000   
     WY                              0.16666667      0.05555556   

            Very Unhealthy Days  Hazardous Days      Max AQI  \
Year State                                                     
2010 AK              0.00000000      0.00000000  88.50000000   
     AL              0.00000000      0.00000000 112.26315789   
     AR              0.00000000      0.00000000 100.00000000   
     AZ              0.00000000      0.15384615 233.46153846   
     CA              0.47169811      0.43396226 305.60377358   
...                         ...             ...          ...   
2016 VT              0.00000000      0.00000000  83.00000000   
     WA              0.00000000      0.00000000  88.17241379   
     WI              0.00000000      0.00000000 118.59259259   
     WV              0.00000000      0.00000000  95.85714286   
     WY              0.00000000      0.00000000  94.77777778   

            90th Percentile AQI  Median AQI    Days CO   Days NO2  \
Year State                                                          
2010 AK             44.87500000 19.75000000 2.37500000 0.00000000   
     AL             70.31578947 46.15789474 0.05263158 0.00000000   
     AR             63.94117647 43.70588235 0.00000000 2.94117647   
     AZ             70.38461538 42.92307692 0.00000000 5.61538462   
     CA             80.60377358 45.30188679 0.15094340 8.90566038   
...                         ...         ...        ...        ...   
2016 VT             42.75000000 29.00000000 0.00000000 3.00000000   
     WA             44.24137931 24.10344828 0.03448276 4.03448276   
     WI             57.55555556 35.96296296 0.00000000 1.33333333   
     WV             52.57142857 35.42857143 4.92857143 0.00000000   
     WY             51.05555556 37.27777778 0.00000000 1.77777778   

             Days Ozone   Days PM2.5   Days PM10  
Year State                                        
2010 AK     59.37500000 191.00000000  9.37500000  
     AL    131.05263158 142.26315789  8.89473684  
     AR    110.35294118  94.94117647  0.00000000  
     AZ    191.23076923  22.76923077 89.92307692  
     CA    222.54716981 105.94339623 12.84905660  
...                 ...          ...         ...  
2016 VT    217.00000000  81.50000000  0.00000000  
     WA     53.34482759 287.44827586  3.89655172  
     WI    195.40740741  85.88888889  0.29629630  
     WV    160.50000000  83.21428571 23.92857143  
     WY    267.05555556  16.05555556 44.50000000  

[35

In [33]:
# result = pd.merge(df_combined, df_ev2, on=["Year"])
result = pd.concat([df_combined, df_ev2], axis=1, join="inner")
result

Days with AQI    Good Days  Moderate Days  \
Year                                                 
2010 AK   262.12500000 222.00000000    35.00000000   
     AL   282.26315789 164.89473684   113.05263158   
     AR   208.23529412 138.52941176    67.35294118   
     AZ   309.53846154 199.07692308   102.23076923   
     CA   350.39622642 221.94339623   102.33962264   
...                ...          ...            ...   
2016 VT   301.50000000 279.75000000    21.50000000   
     WA   348.75862069 317.44827586    30.34482759   
     WI   282.92592593 243.25925926    35.85185185   
     WV   272.57142857 237.78571429    34.07142857   
     WY   329.38888889 286.22222222    42.94444444   

         Unhealthy for Sensitive Groups Days  Unhealthy Days  \
Year                                                           
2010 AK                           4.12500000      1.00000000   
     AL                           4.00000000      0.31578947   
     AR                           2.23529412      0.11764706   
     AZ                           7.92307692      0.15384615   
     CA                          19.88679245      5.32075472   
...                                      ...             ...   
2016 VT                           0.25000000      0.00000000   
     WA                           0.79310345      0.17241379   
     WI                           3.55555556      0.25925926   
     WV                           0.71428571      0.00000000   
     WY                           0.16666667      0.05555556   

         Very Unhealthy Days  Hazardous Days      Max AQI  \
Year                                                        
2010 AK           0.00000000      0.00000000  88.50000000   
     AL           0.00000000      0.00000000 112.26315789   
     AR           0.00000000      0.00000000 100.00000000   
     AZ           0.00000000      0.15384615 233.46153846   
     CA           0.47169811      0.43396226 305.60377358   
...                      ...             ...          ...   
2016 VT           0.00000000      0.00000000  83.00000000   
     WA           0.00000000      0.00000000  88.17241379   
     WI           0.00000000      0.00000000 118.59259259   
     WV           0.00000000      0.00000000  95.85714286   
     WY           0.00000000      0.00000000  94.77777778   

         90th Percentile AQI  Median AQI    Days CO   Days NO2   Days Ozone  \
Year                                                                          
2010 AK          44.87500000 19.75000000 2.37500000 0.00000000  59.37500000   
     AL          70.31578947 46.15789474 0.05263158 0.00000000 131.05263158   
     AR          63.94117647 43.70588235 0.00000000 2.94117647 110.35294118   
     AZ          70.38461538 42.92307692 0.00000000 5.61538462 191.23076923   
     CA          80.60377358 45.30188679 0.15094340 8.90566038 222.54716981   
...                      ...         ...        ...        ...          ...   
2016 VT          42.75000000 29.00000000 0.00000000 3.00000000 217.00000000   
     WA          44.24137931 24.10344828 0.03448276 4.03448276  53.34482759   
     WI          57.55555556 35.96296296 0.00000000 1.33333333 195.40740741   
     WV          52.57142857 35.42857143 4.92857143 0.00000000 160.50000000   
     WY          51.05555556 37.27777778 0.00000000 1.77777778 267.05555556   

          Days PM2.5   Days PM10      value  
Year                                         
2010 AK 191.00000000  9.37500000 0.00000000  
     AL 142.26315789  8.89473684 0.00000000  
     AR  94.94117647  0.00000000 0.00000000  
     AZ  22.76923077 89.92307692 0.00000000  
     CA 105.94339623 12.84905660 0.00000000  
...              ...         ...        ...  
2016 VT  81.50000000  0.00000000 0.00060890  
     WA 287.44827586  3.89655172 0.00097712  
     WI  85.88888889  0.29629630 0.00095506  
     WV  83.21428571 23.92857143 0.00041507  
     WY  16.05555556 44.50000000 0.00015999  

[350 rows x 16 columns]

In [34]:
result.to_csv("analysis.csv", sep='\t', encoding='utf-8')
